<a href="https://colab.research.google.com/github/SuYenTing/d4sg_cipas/blob/main/cipas_NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CKIP實體辨識

2022/02/06 蘇彥庭

參考文件:
[Github: ckiptagger](https://github.com/ckiplab/ckiptagger)

In [ ]:
# 掛載雲端硬碟
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 安裝套件
!pip install -U ckiptagger

# 下載模型
from ckiptagger import data_utils
data_utils.download_data_gdown("./")

Downloading...
From: https://drive.google.com/uc?id=1efHsY16pxK0lBD2gYCgCTnv1Swstq771
To: /content/data.zip
100%|██████████| 1.88G/1.88G [00:06<00:00, 271MB/s]


In [ ]:
# 載入套件
import numpy as np
import pandas as pd
import pickle
import re
from tqdm import tqdm
from ckiptagger import construct_dictionary, WS, POS, NER
# from gensim.models import Word2Vec

In [ ]:
# 讀取資料
cipasWebContentData = pd.read_csv('/content/drive/MyDrive/[不當黨產]/Data/爬蟲資料/cipasWebContentData.csv')
pdfData = pd.read_csv('/content/drive/MyDrive/[不當黨產]/Data/爬蟲資料/pdfData.csv')

In [ ]:
# 取出調查報告文章資料(PDF檔案內容)
reportData = pdfData[(pdfData['source'] == 'news') & (pdfData['attachFileNames'].str.contains('調查報告'))]
reportData = reportData[~(reportData['attachFileNames'].str.contains('PPT|簡報|投影片'))]
reportData = reportData[['source', 'id', 'attachFileNames', 'content']].rename(columns={'attachFileNames': 'articleTitle', 'content': 'articleContent'})

# 取出史料故事文章資料(網站內容)
storiesData = cipasWebContentData[cipasWebContentData['source'] == 'stories'][['source', 'id', 'articleTitle', 'articleContent']]

# 合併資料
docData = pd.concat([reportData, storiesData]).reset_index(drop=True)

In [ ]:
# 載入模型
ws = WS("./data", disable_cuda=False)  # disable_cuda=False: use gpu
pos = POS("./data", disable_cuda=False)
ner = NER("./data", disable_cuda=False)

/usr/local/lib/python3.7/dist-packages/ckiptagger/model_ws.py:106: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class is equivalent as `tf.keras.layers.LSTMCell`, and will be replaced by that in Tensorflow 2.0.
  cell = tf.compat.v1.nn.rnn_cell.LSTMCell(hidden_d, name=name)
/usr/local/lib/python3.7/dist-packages/keras/layers/legacy_rnn/rnn_cell_impl.py:988: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  partitioner=maybe_partitioner)
/usr/local/lib/python3.7/dist-packages/keras/layers/legacy_rnn/rnn_cell_impl.py:996: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  initializer=initializer)
/usr/local/lib/python3.7/dist-packages/ckiptagger/model_pos.py:56: UserWarning: `tf.nn.rnn_cell.LSTMCell` is deprecated and will be removed in a future version. This class

In [ ]:
# 讀取Google雲端sheet的自定義斷詞
customWords = []

sheet_urls = [
              'https://docs.google.com/spreadsheets/d/1ayhAdNJ_Tc9-6ptEkCZSQe-GJ0obpZoAsHvXIUVFOK4/edit#gid=1370388179', 
              'https://docs.google.com/spreadsheets/d/1ayhAdNJ_Tc9-6ptEkCZSQe-GJ0obpZoAsHvXIUVFOK4/edit#gid=725229876',
              ]

for sheet_url in sheet_urls:
    
    sheet_url = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')
    iCustomWords = pd.read_csv(sheet_url, header=None)
    iCustomWords = iCustomWords.to_numpy().flatten()
    iCustomWords = iCustomWords[~pd.isnull(iCustomWords)]
    iCustomWords = list(set(iCustomWords))
    customWords = customWords + iCustomWords

customWords = list(set(customWords))

# 建立自定義詞字典
dictionary = {elem: 1 for elem in customWords}
dictionary = construct_dictionary(dictionary)

In [ ]:
# 清洗資料函數
def cleanText(text):
    # 移除數字, 空格(含跳行), 英文句點, 英文逗點
    # text = re.sub('\d|\s|\.|\,', '', text)

    # 移除空格(含跳行)
    text = re.sub('\s', '', text)
    return text

# 建立儲存表
personDict = dict()  # 儲存人名(PERSON)實體辨識結果
orgDict = dict()  # 儲存組織(ORG)實體辨識結果
wordSentenceList = []  # 儲存各句斷詞後結果

# 迴圈文本資料
# i = 0
for i in tqdm(range(len(docData))):

    # 讀取文本並清洗資料
    doc = docData['articleContent'][i]
    doc = cleanText(doc)

    # 以句號作為句子分隔
    doc = doc.split('。')

    # 執行CKIP斷詞
    word_sentence_list = ws(
        doc,
        sentence_segmentation=True,  # To consider delimiters
        segment_delimiter_set={",", "。", ":", "?", "!", ";"},  # This is the defualt set of delimiters
        coerce_dictionary=dictionary,  # words in this dictionary are forced
        )

    # 儲存斷詞結果
    for word_sentence in word_sentence_list:
        wordSentenceList.append(word_sentence)
        
    # 執行CKIP詞性標註
    pos_sentence_list = pos(word_sentence_list)

    # 執行CKIP實體辨識
    entity_sentence_list = ner(word_sentence_list, pos_sentence_list)

    # 依CKIP實體辨識取出人名(PERSON)與組織(ORG) 並計算該詞出現的次數
    for res in entity_sentence_list:
        for elem in res:
            if elem[2] == 'PERSON':
                if elem[3] not in personDict:  # 若第一次出現的詞則加入字典並初始化次數為1
                    personDict[elem[3]] = 1
                else:
                    personDict[elem[3]] += 1  # 先前已出現過該詞 計數再加1次
            elif elem[2] == 'ORG':
                if elem[3] not in orgDict:
                    orgDict[elem[3]] = 1
                else:
                    orgDict[elem[3]] += 1

100%|██████████| 354/354 [12:14<00:00,  2.08s/it]


In [ ]:
# 將實體辨識結果轉為資料表
personEntityDf = pd.DataFrame(data={'person': personDict.keys(), 'counts': personDict.values()})
personEntityDf = personEntityDf.sort_values(['counts'], ascending=False)  # 案詞頻排序
personEntityDf = personEntityDf[personEntityDf['person'].str.len() > 1]  # 過濾掉只有1個字的詞
personEntityDf = personEntityDf[~personEntityDf['person'].str.contains('○|〇|O|Ｏ')]  # 刪除具遮蔽之名稱
personEntityDf = personEntityDf.reset_index(drop=True)

orgEntityDf = pd.DataFrame(data={'org': orgDict.keys(), 'counts': orgDict.values()})
orgEntityDf = orgEntityDf.sort_values(['counts'], ascending=False)
orgEntityDf = orgEntityDf[orgEntityDf['org'].str.len() > 1]
orgEntityDf = orgEntityDf.reset_index(drop=True)

In [ ]:
# 輸出結果
personEntityDf.to_csv('personEntityDf.csv', index=False, encoding='utf-8-sig')
orgEntityDf.to_csv('orgEntityDf.csv', index=False, encoding='utf-8-sig')

In [ ]:
# # 儲存斷詞後結果
# with open('/content/drive/MyDrive/d4sg/doc_ws.pickle', 'wb') as f:
#     pickle.dump([wordSentenceList, personEntityDf, orgEntityDf], f)

# 讀取斷詞後結果
with open('/content/drive/MyDrive/d4sg/doc_ws.pickle', 'rb') as f:
    wordSentenceList, personEntityDf, orgEntityDf = pickle.load(f)

In [ ]:
# 處理姓名及職稱問題
forPersonJobEntityDf = personEntityDf[personEntityDf['person'].str.len() >= 1]

# 處理姓氏
def lastName(person):
    if len(person) >= 4:  # 名字4個字以上 則姓氏取前2個字
        return person[0:2]
    else:
        return person[0]

# 處理名字
def firstName(person):
    if len(person) >= 4:  # 名字4個字以上 則名字取第二個字以後
        return person[2:]
    else:
        return person[1:]

forPersonJobEntityDf['lastName'] = personEntityDf['person'].apply(lastName)
forPersonJobEntityDf['firstName'] = personEntityDf['person'].apply(firstName)

In [ ]:
# 建立姓氏+職稱+名字儲存表
matchPersonJob = list()

# sentence = wordSentenceList[8284]
# 迴圈各句比對是否有姓氏+職稱+名字儲存表
for i in tqdm(range(len(wordSentenceList))):

    sentence = wordSentenceList[i]

    # 判斷該詞是否有符合的名字 若有符合則再對應姓氏 若皆符合名字及姓氏 則儲存姓氏+職稱+名字
    for idx, value in enumerate(sentence):
        if idx > 1:
            matchLastName = forPersonJobEntityDf[forPersonJobEntityDf['firstName'] == value]['lastName'].tolist()
            if sentence[idx-2] in matchLastName:
                matchPersonJob.append(sentence[(idx-2):(idx+1)])

100%|██████████| 20365/20365 [06:42<00:00, 50.58it/s]


In [ ]:
# 將姓氏+職稱+名字整理成資料表格
personJobEntityDf = pd.DataFrame(matchPersonJob, columns=['lastName', 'jobTitle', 'firstName'])
personJobEntityDf = personJobEntityDf.drop_duplicates()
personJobEntityDf = personJobEntityDf.sort_values('lastName', ascending=False).reset_index(drop=True)

In [ ]:
# 輸出結果
personJobEntityDf.to_csv('personJobEntityDf.csv', index=False, encoding='utf-8-sig')